<a href="https://colab.research.google.com/github/m-bashari-m/vehicle-color-recognition/blob/main/src/02_rgb_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Prepare utils file

!wget https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py

--2022-06-25 08:36:27--  https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12099 (12K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]  11.82K  --.-KB/s    in 0s      

2022-06-25 08:36:27 (51.2 MB/s) - ‘utils.py’ saved [12099/12099]



In [3]:
from google.colab import drive
drive.mount('./drive')

Mounted at ./drive


In [4]:
import pandas as pd
import numpy as np
import os

import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub

from utils import ModelCreator, get_train_val_ds, ErrorAnalyzer, get_class_weight

In [5]:
BATCH_SIZE = 32
IMG_SIZE = (224, 224)
AUTOTUNE = tf.data.AUTOTUNE
N_CLASSES = 16
HUB_URL = 'https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_b3/feature_vector/2'

In [6]:
dataset_dir = os.path.join('drive', 'MyDrive', 'cars')
train_dir = os.path.join(dataset_dir, 'train')
val_dir = os.path.join(dataset_dir, 'val')

In [7]:
# Get shuffled train and validation dataset with one-hot labels

train_ds, val_ds = get_train_val_ds(train_dir, val_dir, batch_size=BATCH_SIZE, img_size=IMG_SIZE)

Found 16580 files belonging to 16 classes.
Found 3508 files belonging to 16 classes.


In [8]:
# Get class weights which have calculated from the previous notebook and stored in github

classes, class_weight = get_class_weight()

In [ ]:
for color, weight in zip(classes, class_weight.values()):
  print(color, "    \t=>", weight)

beige     	=> 17.94
black     	=> 13.8
blue     	=> 15.47
brown     	=> 17.92
cream     	=> 56.81
crimson     	=> 76.43
gold     	=> 56.81
green     	=> 21.2
grey     	=> 12.41
navy-blue     	=> 58.77
orange     	=> 22.37
red     	=> 16.39
silver     	=> 20.17
titanium     	=> 56.81
white     	=> 2.97
yellow     	=> 20.68


In [9]:
train_ds = (
    train_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

val_ds = (
    val_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

In [10]:
rgb_creator = ModelCreator(HUB_URL, 'RGB-model')
rgb_model = rgb_creator.make_model(img_size=IMG_SIZE)

Model: "RGB-model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1536)              12930622  
                                                                 
 dense (Dense)               (None, 16)                24592     
                                                                 
Total params: 12,955,214
Trainable params: 12,845,998
Non-trainable params: 109,216
_________________________________________________________________


In [11]:
# Callbacks are EarlyStopping and ModelCheckpoint

callbacks = rgb_creator.get_callbacks()

In [12]:
history = rgb_model.fit(train_ds,
                        callbacks=callbacks,
                        epochs=15,
                        validation_data=val_ds,
                        class_weight=class_weight)

Epoch 1/15
519/519 [==============================] - 2636s 5s/step - loss: 14.9032 - auc: 0.7918 - accuracy: 0.7065 - val_loss: 1.0909 - val_auc: 0.6769 - val_accuracy: 0.6212
Epoch 2/15
519/519 [==============================] - 191s 365ms/step - loss: 8.9283 - auc: 0.8978 - accuracy: 0.8128 - val_loss: 0.8043 - val_auc: 0.8214 - val_accuracy: 0.7289
Epoch 3/15
519/519 [==============================] - 184s 352ms/step - loss: 6.6187 - auc: 0.9327 - accuracy: 0.8605 - val_loss: 1.7000 - val_auc: 0.5446 - val_accuracy: 0.5205
Epoch 4/15
519/519 [==============================] - 185s 354ms/step - loss: 5.5756 - auc: 0.9380 - accuracy: 0.8710 - val_loss: 0.7676 - val_auc: 0.8625 - val_accuracy: 0.8002
Epoch 5/15
519/519 [==============================] - 185s 355ms/step - loss: 3.8524 - auc: 0.9675 - accuracy: 0.9106 - val_loss: 0.6678 - val_auc: 0.8889 - val_accuracy: 0.8235
Epoch 6/15
519/519 [==============================] - 190s 363ms/step - loss: 3.2511 - auc: 0.9731 - accuracy: 

In [15]:
history = rgb_model.fit(train_ds,
                        callbacks=callbacks,
                        epochs=20,
                        initial_epoch=10,
                        validation_data=val_ds,
                        class_weight=class_weight)

Epoch 11/20
519/519 [==============================] - 189s 363ms/step - loss: 0.7564 - auc: 0.9967 - accuracy: 0.9817 - val_loss: 1.0689 - val_auc: 0.8392 - val_accuracy: 0.8170
Epoch 12/20
519/519 [==============================] - 185s 355ms/step - loss: 0.7321 - auc: 0.9970 - accuracy: 0.9846 - val_loss: 1.1278 - val_auc: 0.8477 - val_accuracy: 0.8355
Epoch 13/20
519/519 [==============================] - 185s 354ms/step - loss: 0.5081 - auc: 0.9975 - accuracy: 0.9874 - val_loss: 1.1698 - val_auc: 0.8339 - val_accuracy: 0.8213
Epoch 14/20
519/519 [==============================] - 185s 354ms/step - loss: 0.4426 - auc: 0.9973 - accuracy: 0.9879 - val_loss: 1.3878 - val_auc: 0.8044 - val_accuracy: 0.8039
Epoch 15/20
519/519 [==============================] - 190s 364ms/step - loss: 0.3734 - auc: 0.9985 - accuracy: 0.9911 - val_loss: 1.2638 - val_auc: 0.8245 - val_accuracy: 0.8067
Epoch 16/20
519/519 [==============================] - 190s 364ms/step - loss: 0.5160 - auc: 0.9966 - acc

In [16]:
!cp ./logs/checkpoints/RGB-model.h5 ./drive/MyDrive/checkpoints/rgb-model-final.h5